In [1]:
import gradio as gr
import numpy as np
import pandas as pd

import os
import time
import random
import math

import matplotlib.pyplot as plt

import copy

In [2]:
print(gr.__version__)

4.8.0


In [3]:
def random_values_generate():
    random_int = random.randint(0, 13)
    return random_int

In [4]:
def get_llm_output(input_query, is_demo = True):

    if is_demo:
        _text =  "This is a demo output"
    else:
        pass
        # _text = gqws.get_qa_w_sources(input_query, is_demo = False)

    return _text


def get_summary():
    is_demo=True
    if is_demo:
        _cache = ""
        _text =  """
## **Action Plan**

**1. Immediate Attention to Vital Signs**
- Monitor the patient's vitals consistently, considering the concerning sepsis percentage (83%) and septic shock percentage (63%). Given the fluctuation in respiration, consider closer scrutiny.

**2. Cardiac Assessment**
- Prioritize evaluation of the cardiovascular system. The patient has been diagnosed with cardiogenic shock, congestive heart failure, and cardiac arrest, suggesting critical cardiovascular compromise.

**3. Pulmonary Focus**
- The patient's respiratory status is vital, as evidenced by diagnoses like hypoxemia and acute respiratory failure. Consider aggressive respiratory support, potential intubation if required.

**4. Review of Medications**
- With the presence of cardiogenic shock and cardiac arrest, review the current medication list for possible contraindications or required alterations.

**5. Consultation and Collaboration**
- Engage with a cardiologist and pulmonologist for specialized input.

**References**
- Bhatti, A., Thangavelu, N., Hassan, M., Kim, C., Lee, S., Kim, Y., & Kim, J. Y. (2023). Interpreting Forecasted Vital Signs Using N-BEATS in Sepsis Patients. *arXiv:2306.14016*. [Link](https://doi.org/10.48550/arXiv.2306.14016). Accepted in 1st World Conference on eXplainable Artificial Intelligence - Late-breaking work, Demos and Doctoral Consortium, 2023.
"""

        for words in _text.split(" "):
            delay_words = np.random.randint(5, 10)/100
            time.sleep(delay_words)
            for _c in words:
                delay_chars = np.random.randint(2, 6)/100
                time.sleep(delay_chars)
                _cache += _c
                yield _cache
            _cache += " "

    # else:
    #     return get_llm_output(input_query, is_demo = False)   

In [5]:
plot_end = 108

state_dict = {
    0: "Blood Pressure",
    1: "Heart Rate",
    2: "Respiration",
}

plt.style.use('ggplot')

_data_df = pd.read_csv("./merged_df_sample.csv")

def get_patient_data():    
    """returns 4x3 table with mean, max, min, std for vitals"""
    # global CURRENT_TIMESTAMP
    # global INITIAL_TIMESTAMP

    is_demo = True

    if is_demo:
        pid = _data_df["patientunitstayid"].unique()[0]
        #print(is_demo, pid)

    # if CURRENT_TIMESTAMP >= MAX_TIMESTAMP:
    #     CURRENT_TIMESTAMP = INITIAL_TIMESTAMP

    patient_vitals_df = _data_df[_data_df["patientunitstayid"] == pid]
    # mean bp: MBP = diastolic blood pressure (DBP) + 1/3 [systolic blood pressure (SBP) – DBP]
    patient_vitals_df["mean_bp"] = patient_vitals_df["diastolicbp"] + (1/3)*(patient_vitals_df["systolicbp"] - patient_vitals_df["diastolicbp"])

    return patient_vitals_df



def get_plot(plot_type): ## variable can be added to update the generated plot! e.g. high chance of sepsis percentage can be used to manipulate the plot.
    global plot_end

    # rnd_value = random_values_generate()

    data_df = get_patient_data()
    
    x = np.arange(plot_end - 108, plot_end, 1)
    df = data_df.iloc[plot_end - 108:plot_end].copy()
    
    for _col in ["mean_bp", "heartrate", "respiration"]:
            _random_multipliar = np.concatenate([np.ones(108-36), 1 + np.clip(np.random.normal(0, 0.01, 36), a_min=-0.05, a_max=0.05)])
            df[_col] = df[_col] * _random_multipliar

    # forecast_noise = np.array([1 + random.randint(-5, 5)/100. for x in range(36)])
    # forecaste_index = plot_end - 36

    bp = df['mean_bp'].rolling(3).mean().values ### update this function! vital signs data should be added here.
    hr = df['heartrate'].rolling(3).mean().values ### update this function! vital signs data should be added here.
    rp = df['respiration'].rolling(3).mean().values ### update this function! vital signs data should be added here.

    data=pd.DataFrame({"x": x, "bp": bp, "hr": hr, "rp": rp})

    # 1,1 plot on ax
    plt.style.use('ggplot')
    plt.grid(True)

    fig1, ax = plt.subplots(3, 1, figsize=(12, 7))

    # df_forecast = data.iloc[71:108].copy()

    # plot bp, hr, rp
    ax[0].plot(data["x"][:72], data['bp'][:72].to_numpy(), color='blue', label='Past')
    ax[0].plot(data["x"][71:108], data['bp'][71:108].to_numpy(), color='red', label='Future')

    # plot standard deviation as shaded area for forecast/Future
    ax[0].fill_between(data["x"][71:108], data['bp'][71:108].to_numpy() - 0.01 * data['bp'][71:108].to_numpy(), \
                       data['bp'][71:108].to_numpy() + 0.01 * data['bp'][71:108].to_numpy(), alpha=0.2, color='red')

    ax[1].plot(data["x"][:72], data['hr'][:72].to_numpy(), color='blue', label='Past')
    ax[1].plot(data["x"][71:108], data['hr'][71:108].to_numpy(), color='red', label='Future')

    # plot standard deviation as shaded area for forecast/Future
    ax[1].fill_between(data["x"][71:108], data['hr'][71:108].to_numpy() - 0.01 * data['hr'][71:108].to_numpy(), \
                       data['hr'][71:108].to_numpy() + 0.01 * data['hr'][71:108].to_numpy(), alpha=0.2, color='red')

    ax[2].plot(data["x"][:72], data['rp'][:72].to_numpy(), color='blue', label='Past')
    ax[2].plot(data["x"][71:108], data['rp'][71:108].to_numpy(), color='red', label='Future')

    # plot standard deviation as shaded area for forecast/Future
    ax[2].fill_between(data["x"][71:108], data['rp'][71:108].to_numpy() - 0.15 * data['rp'][71:108].to_numpy(), \
                       data['rp'][71:108].to_numpy() + 0.23 * data['rp'][71:108].to_numpy(), alpha=0.2, color='red')

    #plot vertical line on all subplots
    ax[0].axvline(x=data["x"].max() - 36, linestyle='--', lw=1.5)
    ax[1].axvline(x=data["x"].max() - 36, linestyle='--', lw=1.5)
    ax[2].axvline(x=data["x"].max() - 36, linestyle='--', lw=1.5)


    # y label for ax[0], ax[1], ax[2]
    ax[0].set_ylabel("Mean Blood Pressure")
    ax[1].set_ylabel("Heart Rate")
    ax[2].set_ylabel("Respiration")

    # add legend to the plot
    ax[0].legend()
    ax[1].legend()
    ax[2].legend()

    # add grid to the plot
    ax[0].grid()
    ax[1].grid()
    ax[2].grid()    

    # add tight layout to the plot
    # plt.tight_layout()

    # add padding to the plot
    # plt.subplots_adjust(top=0.88)

    plot_end += 1
    if plot_end > 2000:
        plot_end = 108

    plt.close()
    return fig1

FileNotFoundError: [Errno 2] No such file or directory: './merged_df_sample.csv'

In [ ]:
# data_df = get_patient_vitals()

# data_df

In [ ]:
# summary_patient = '''| **Section**                  | **Details**                                                                                                                                                           |
# |------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------|
# | **ID**                       | Name: Not mentioned <br> Age: Not mentioned <br> Gender: Not mentioned                                                                                                 |
# | **REASON FOR VISIT**         | Leg pain and difficulty walking                                                                                                                                      |
# | **PAST MEDICAL HISTORY**     | Borderline diabetes for 20 years                                                                                                                                     |
# | **HOME MEDICATIONS**         | Diabetes medication (not mentioned)                                                                                                                                  |
# | **ALLERGIES**                | Shellfish                                                                                                                                                             |
# | **FAMILY HISTORY**           | Multiple family members died in their sleep                                                                                                                          |
# | **SOCIAL HISTORY**           | • Smoking: 1-2 packs per day <br> • Alcohol: Not mentioned <br> • Occupation: Not mentioned                                                                           |
# | **HISTORY OF PRESENT ILLNESS** | The patient has been experiencing difficulty walking for 20 years, with pain in the left leg, starting in the ankle and calf, and sometimes affecting both legs. The pain is relieved by rest. The patient also experiences numbness in the toes and fingers, and sometimes wakes up at night due to numbness and tingling sensations. The patient denies chest pain but reports heartburn sensations after walking. The patient has shortness of breath but it is not the primary reason for stopping walking. |
# | **PHYSICAL EXAM**             | Not mentioned                                                                                                                                                        |
# '''

summary_patient = '''## **Profile**
- **Name:** John Doe
- **Gender:** Male
- **Age:** 41
- **Ethnicity:** Caucasian
- **Location:** Intensive Care Unit

## **ICU Admission Reason**
Admitted due to symptoms indicative of cardiac and respiratory disturbances.

## **Historical Medical Data**
- **Cardiovascular Issues:** Recorded instances of ventricular disorders, congestive heart failure, cardiogenic shock, and cardiac arrest.
- **Pulmonary Concerns:** Episodes of respiratory failure, characterized by hypoxemia and acute respiratory distress.
  
## **Vital Signs**
- **Blood Pressure:** Demonstrated fluctuations, with means around 89-90 mmHg.
- **Heart Rate:** Elevated rates averaging around 120-123 bpm, with highs reaching 127 bpm.
- **Respiration:** Rates varied widely, from as low as 12/min to as high as 81/min.

## **Sepsis & Shock Metrics**
- **Sepsis Risk:** 83% 
- **Septic Shock Risk:** 63%

## **Additional Information**
- **Allergies:** To be determined.
- **Family History:** Need details, especially regarding cardiovascular issues.
- **Social Habits:** Full assessment required to understand potential risk factors.
'''

In [ ]:
state_dict = {
    0: "Blood Pressure",
    1: "Heart Rate",
    2: "Respiration",
}

glb_sepsis = 67
glb_shock = 45

_data_df = pd.read_csv("./merged_df_sample.csv")

INITIAL_TIMESTAMP = 36
CURRENT_TIMESTAMP = 36
TIMESTAMP_INCREMENT = 1
MAX_TIMESTAMP = len(_data_df)
WINDOW_SIZE_3HR = 36
WINDOW_SIZE_30MIN = 6

patient_info_cols = ["gender", "age", "ethnicity", "admissionweight", "hospitaladmittime24"]
patient_vitals_columns = ["groups", "patientunitstayid", "observationoffset", "systolicbp", "diastolicbp", "heartrate", "respiration"]

def get_patient_info():
    """takes patient id and returns dataframe of patient info"""
    is_demo = True

    if is_demo:
        pid = _data_df["patientunitstayid"].unique()[0]

    patient_info_static_df = _data_df[_data_df["patientunitstayid"] == pid] ## dataframme with static patient info

    patient_info_static_df = patient_info_static_df.iloc[0][patient_info_cols]
    patient_info_static_df["Name"] = "John Doe"
    patient_info_static_df = patient_info_static_df[["Name"] + patient_info_cols]

    patient_info_static_df.rename(index={"gender": "Sex", "age": "Age", "ethnicity": "Ethnicity", "admissionweight": "Weight (kg.)", "hospitaladmittime24": "Admission Time"}, inplace=True)
    patient_info_static_df = patient_info_static_df.reset_index()

    patient_info_static_df.columns = ["Info", "Value"]
    return patient_info_static_df ## dataframe with patient_info_cols + Name

def get_patient_vitals():
    """returns 4x3 table with mean, max, min, std for vitals"""
    global CURRENT_TIMESTAMP
    global INITIAL_TIMESTAMP

    is_demo = True

    if is_demo:
        pid = _data_df["patientunitstayid"].unique()[0]
        #print(is_demo, pid)

    if CURRENT_TIMESTAMP >= MAX_TIMESTAMP:
        CURRENT_TIMESTAMP = INITIAL_TIMESTAMP

    patient_vitals_df = _data_df[_data_df["patientunitstayid"] == pid]
    # mean bp: MBP = diastolic blood pressure (DBP) + 1/3 [systolic blood pressure (SBP) – DBP]
    patient_vitals_df["mean_bp"] = patient_vitals_df["diastolicbp"] + (1/3)*(patient_vitals_df["systolicbp"] - patient_vitals_df["diastolicbp"])

    # 3hr = 36 rows
    patient_vitals_df_window_1 = patient_vitals_df.iloc[CURRENT_TIMESTAMP-WINDOW_SIZE_3HR:CURRENT_TIMESTAMP][patient_vitals_columns + ["mean_bp"]]

    patient_vitals_df_window_1 = patient_vitals_df_window_1.sort_values(by="observationoffset", ascending=False)[["mean_bp", "heartrate", "respiration"]].agg(["mean", "max", "min", "std"]).reset_index()
    patient_vitals_df_window_1.columns = ["Stats 3-Hrs", "Mean BP (mmHg)", "Heartrate (bpm)", "Respiration (/min.)"]
    patient_vitals_df_window_1["Stats 3-Hrs"] = np.array(["Mean", "Max.", "Min.", "Std."])

    # add window size

    # 30 mins = 6 rows
    patient_vitals_df_window_2 = patient_vitals_df.iloc[CURRENT_TIMESTAMP-WINDOW_SIZE_30MIN:CURRENT_TIMESTAMP][patient_vitals_columns + ["mean_bp"]]
    patient_vitals_df_window_2 = patient_vitals_df_window_2.sort_values(by="observationoffset", ascending=False)[["mean_bp", "heartrate", "respiration"]].agg(["mean", "max", "min", "std"]).reset_index()
    
    patient_vitals_df_window_2.columns = ["Stats 30-Mins", "Mean BP (mmHg)", "Heartrate (bpm)", "Respiration (/min.)"]
    patient_vitals_df_window_2["Stats 30-Mins"] = np.array(["Mean", "Max.", "Min.", "Std."])

    CURRENT_TIMESTAMP += TIMESTAMP_INCREMENT

    return patient_vitals_df_window_1.round(2), patient_vitals_df_window_2.round(2)

def sepsis_label(period=0):
    global glb_sepsis
    global glb_shock

    if glb_sepsis >= 85:
        glb_sepsis = 84
    if glb_sepsis <= 16:
        glb_sepsis = 60

    random_sepsis_number = random.randint(glb_sepsis-15, glb_sepsis+15)
    
    if random_sepsis_number > 80:
        random_shock_number = random.randint(55, random_sepsis_number)
    else: random_shock_number = random.randint(0, random_sepsis_number)

    # update global variables
    glb_sepsis = random_sepsis_number
    glb_shock = random_shock_number

    sepsis = {"Sepsis": random_sepsis_number/100., "Septic Shock": random_shock_number/100.} ### update this function! sepsis and septic shock percentage should be added here.
    
    return sepsis

def row_textbox(label):
    with gr.Row() as row_textbox:
        gr.Textbox(lines=1, label=label)
    return row_textbox

def row_doc_report(label):
    with gr.Row() as row_report:
            gr.Textbox(lines=6, label="Summary", show_copy_button=True)
            # text_box_info.submit(get_summary, inputs = [text_box_info], outputs = [text_box_summary])
    return row_report

def feedback_reset():
    # feedback_button.value = None
    # return ["### Feedback Submitted! Thank You"]
    return "<h2 style='text-align: center; color: green;'>Feedback Submitted! Thank You</h2>"

with gr.Blocks() as demo:
    
    with gr.Row():
        # with gr.Blocks(height=100) as timeseries:
        with gr.Column(scale=3):
            gr.Markdown("## Vital Signs")
            bp_plot = gr.Plot(show_label=False)

        with gr.Column(scale=1):
            gr.Markdown("## Patient Profile")
            with gr.Row():                
                # gr.Image("/home/bhatti/Personal_llms/LLM/abhatti/source/Images/blank_user.png", height=240, width=50)
                info_df = gr.Dataframe(col_count=2, interactive=False) # , scale=1
            # gr.Markdown("## Sepsis and Septic Shock Meter")
            with gr.Row():
                sep_label = gr.Label(label="Sepsis & Shock Meter", show_label=True)                

    with gr.Row():
        with gr.Blocks():
            with gr.Row():
                # pid = gr.Textbox(lines=1, label="Patient ID", placeholder="Patient ID or `demo`", interactive=True, show_copy_button=True, value="demo", visible=False)
                vital_stat_df_window_1 = gr.Dataframe(label="Stats 3 Hr", col_count=4, scale=2, interactive=False)
                vital_stat_df_window_2 = gr.Dataframe(label="Stats 30 Mins", col_count=4, scale=2, interactive=False)

    with gr.Blocks(css = "border: 1px solid black; padding: 10px;"):

        with gr.Row():
            with gr.Column(scale = 2):
                summary = gr.Markdown(label="Patient History", value=summary_patient)
                summary_button = gr.Button(value="Get Report")

            with gr.Column(scale = 2):
                with gr.Row():
                    feedback_button = gr.Radio(["Yes", "No"], label="Is this Treatment Plan correct?")
                    feedback_submit_button = gr.Button(value="Submit Feedback")

                summary_model = gr.Markdown(label="Treatment Plan")
                feedback_msg = gr.Markdown(label="Feedback")
                
    # timeseries.load()
    state_bp = gr.Number(value=0, visible=False)

    demo.load(get_plot, state_bp, bp_plot, every=10)
    demo.load(sepsis_label, None, sep_label, every=10)

    demo.load(get_patient_info, every=2, outputs=[info_df])
    demo.load(get_patient_vitals, every=2, outputs=[vital_stat_df_window_1, vital_stat_df_window_2])

    summary_button.click(get_summary, outputs = [summary_model])
    feedback_submit_button.click(feedback_reset, outputs = [feedback_msg])
    
demo.queue().launch(share=True, height=600, width=600, max_threads=2)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://3dc0a280d5ed9b731d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
